In [3]:
from datasets import load_dataset

In [4]:
squad_it_dataset = load_dataset("json", data_files="./data/SQuAD_it-train.json", field="data")

In [5]:
data_files = {"train": "./data/SQuAD_it-train.json", "test": "./data/SQuAD_it-test.json"}
squad_it_dataset = load_dataset("json", data_files=data_files, field="data")

In [6]:
squad_it_dataset

DatasetDict({
    train: Dataset({
        features: ['title', 'paragraphs'],
        num_rows: 442
    })
    test: Dataset({
        features: ['title', 'paragraphs'],
        num_rows: 48
    })
})

In [7]:
# try to load a dataset from UCI
farm_ads_url = "https://archive.ics.uci.edu/static/public/218/farm+ads.zip"
farm_ads_dataset = load_dataset("text", data_files=farm_ads_url)

Generating train split: 0 examples [00:00, ? examples/s]

In [12]:
farm_ads_dataset['train']['text'][1000:1005]

['1 ad-soap ad-japan ad-charcoal ad-soap ad-hinoki ad-oil ad-natural ad-antibacterial ad-effect ad-www ad-goldsakura ad-com title-natural title-beauty title-product title-cosmetic title-beauty title-product title-japanese title-skin title-care title-product title-goldsakura header-sel header-product header-natural header-beauty header-product header-cosmetic header-beauty header-product header-japanese header-skin header-care header-product natural beauty product cosmetic beauty product japanese skin care product goldsakura skip main content authentic japanese cosmetics account cart checkout log search site search select language category navigation natural cleanser product wash product exfoliate deep cleanse remove makeup tone moisturize sun protection skin care product body body care product hand care product set hair collection hanayuki hada moist hinoki deitanseki hinoki deitanseki gift set ingredient sel product natural beauty product cosmetic beauty product japanese skin care pro

In [ ]:
!wget "https://archive.ics.uci.edu/ml/machine-learning-databases/00462/drugsCom_raw.zip" -P ./data

--2025-05-25 14:27:11--  https://archive.ics.uci.edu/ml/machine-learning-databases/00462/drugsCom_raw.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘./data/drugsCom_raw.zip’

drugsCom_raw.zip        [                 <=>]  41.00M  1.41MB/s    in 25s     

2025-05-25 14:27:37 (1.62 MB/s) - ‘./data/drugsCom_raw.zip’ saved [42989872]

Archive:  ./data/drugsCom_raw.zip
replace drugsComTest_raw.tsv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [16]:
!unzip ./data/drugsCom_raw.zip -d ./data

Archive:  ./data/drugsCom_raw.zip
  inflating: ./data/drugsComTest_raw.tsv  
  inflating: ./data/drugsComTrain_raw.tsv  


In [17]:
data_files = {"train": "./data/drugsComTrain_raw.tsv", "test": "./data/drugsComTest_raw.tsv"}
drug_dataset = load_dataset("csv", data_files=data_files, delimiter="\t")

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [18]:
drug_sample = drug_dataset["train"].shuffle(seed=42).select(range(1000))

In [19]:
drug_sample[:3]

{'Unnamed: 0': [87571, 178045, 80482],
 'drugName': ['Naproxen', 'Duloxetine', 'Mobic'],
 'condition': ['Gout, Acute', 'ibromyalgia', 'Inflammatory Conditions'],
 'review': ['"like the previous person mention, I&#039;m a strong believer of aleve, it works faster for my gout than the prescription meds I take. No more going to the doctor for refills.....Aleve works!"',
  '"I have taken Cymbalta for about a year and a half for fibromyalgia pain. It is great\r\nas a pain reducer and an anti-depressant, however, the side effects outweighed \r\nany benefit I got from it. I had trouble with restlessness, being tired constantly,\r\ndizziness, dry mouth, numbness and tingling in my feet, and horrible sweating. I am\r\nbeing weaned off of it now. Went from 60 mg to 30mg and now to 15 mg. I will be\r\noff completely in about a week. The fibro pain is coming back, but I would rather deal with it than the side effects."',
  '"I have been taking Mobic for over a year with no side effects other than 

In [27]:
drug_dataset['train'][1]

{'Unnamed: 0': 95260,
 'drugName': 'Guanfacine',
 'condition': 'ADHD',
 'review': '"My son is halfway through his fourth week of Intuniv. We became concerned when he began this last week, when he started taking the highest dose he will be on. For two days, he could hardly get out of bed, was very cranky, and slept for nearly 8 hours on a drive home from school vacation (very unusual for him.) I called his doctor on Monday morning and she said to stick it out a few days. See how he did at school, and with getting up in the morning. The last two days have been problem free. He is MUCH more agreeable than ever. He is less emotional (a good thing), less cranky. He is remembering all the things he should. Overall his behavior is better. \r\nWe have tried many different medications and so far this is the most effective."',
 'rating': 8.0,
 'date': 'April 27, 2010',
 'usefulCount': 192}

In [22]:
for split in drug_dataset.keys():
    assert len(drug_dataset[split]) == len(drug_dataset[split].unique("Unnamed: 0"))

In [28]:
drug_dataset = drug_dataset.rename_column(
    original_column_name="Unnamed: 0", new_column_name="patient_id"
)

In [35]:
len(drug_dataset['train'])

161297

In [34]:
len(drug_dataset.unique("drugName")['train'])

3436

In [36]:
len(drug_dataset.unique("condition")['train'])

885

In [38]:
def filter_nones(x):
    return x["condition"] is not None

In [39]:
drug_dataset = drug_dataset.filter(lambda x: x["condition"] is not None)

Filter:   0%|          | 0/161297 [00:00<?, ? examples/s]

Filter:   0%|          | 0/53766 [00:00<?, ? examples/s]

In [40]:
len(drug_dataset['train'])

160398

In [43]:
def lowercase_condition(example):
    return {"condition": example["condition"].lower()}


drug_dataset = drug_dataset.map(lowercase_condition)

In [44]:
drug_dataset["train"]["condition"][:3]

['left ventricular dysfunction', 'adhd', 'birth control']

In [45]:
def compute_review_length(example):
    return {"review_length": len(example["review"].split())}

In [57]:
drug_dataset = drug_dataset.map(compute_review_length)

Map:   0%|          | 0/160398 [00:00<?, ? examples/s]

Map:   0%|          | 0/53471 [00:00<?, ? examples/s]

In [58]:
drug_dataset["train"][0]

{'patient_id': 206461,
 'drugName': 'Valsartan',
 'condition': 'left ventricular dysfunction',
 'review': '"It has no side effect, I take it in combination of Bystolic 5 Mg and Fish Oil"',
 'rating': 9.0,
 'date': 'May 20, 2012',
 'usefulCount': 27,
 'review_length': 17}

In [59]:
drug_dataset["train"].sort("review_length")[:3]

{'patient_id': [111469, 13653, 53602],
 'drugName': ['Ledipasvir / sofosbuvir',
  'Amphetamine / dextroamphetamine',
  'Alesse'],
 'condition': ['hepatitis c', 'adhd', 'birth control'],
 'review': ['"Headache"', '"Great"', '"Awesome"'],
 'rating': [10.0, 10.0, 10.0],
 'date': ['February 3, 2015', 'October 20, 2009', 'November 23, 2015'],
 'usefulCount': [41, 3, 0],
 'review_length': [1, 1, 1]}

In [65]:
drug_dataset = drug_dataset.filter(lambda x: x["review_length"] > 30)

Filter:   0%|          | 0/160398 [00:00<?, ? examples/s]

Filter:   0%|          | 0/53471 [00:00<?, ? examples/s]

In [66]:
drug_dataset.num_rows

{'train': 138514, 'test': 46108}

In [69]:
drug_dataset.sort("review_length", reverse=True)['train'][:3]

{'patient_id': [121004, 181160, 216072],
 'drugName': ['Venlafaxine', 'Prozac', 'Copper'],
 'condition': ['migraine', 'obsessive compulsive disorde', 'birth control'],
 'review': ['"Two and a half months ago I was prescribed Venlafaxine to help prevent chronic migraines.\r\nIt did help the migraines (reduced them by almost half), but with it came a host of side effects that were far worse than the problem I was trying to get rid of.\r\nHaving now come off of the stuff, I would not recommend anyone ever use Venlafaxine unless they suffer from extreme / suicidal depression. I mean extreme in the most emphatic sense of the word. \r\nBefore trying Venlafaxine, I was a writer. While on Venlafaxine, I could barely write or speak or communicate at all. More than that, I just didn&#039;t want to. Not normal for a usually outgoing extrovert.\r\nNow, I&#039;m beginning to write again - but my ability to speak and converse with others has deteriorated by about 95%. Writing these words is taking f

In [71]:
import html

text = "I&#039;m a transformer called BERT"
html.unescape(text)

"I'm a transformer called BERT"

In [72]:
drug_dataset = drug_dataset.map(lambda x: {"review": html.unescape(x["review"])})

Map:   0%|          | 0/138514 [00:00<?, ? examples/s]

Map:   0%|          | 0/46108 [00:00<?, ? examples/s]

In [ ]:
# This is supposedly faster
drug_dataset = drug_dataset.map(lambda x: {"review": [html.unescape(o) for o in x["review"]]}, batched=True)

Map:   0%|          | 0/138514 [00:00<?, ? examples/s]

Map:   0%|          | 0/46108 [00:00<?, ? examples/s]

In [84]:
drug_dataset['train'][:3]

{'patient_id': [95260, 92703, 138000],
 'drugName': ['Guanfacine', 'Lybrel', 'Ortho Evra'],
 'condition': ['adhd', 'birth control', 'birth control'],
 'review': ['"My son is halfway through his fourth week of Intuniv. We became concerned when he began this last week, when he started taking the highest dose he will be on. For two days, he could hardly get out of bed, was very cranky, and slept for nearly 8 hours on a drive home from school vacation (very unusual for him.) I called his doctor on Monday morning and she said to stick it out a few days. See how he did at school, and with getting up in the morning. The last two days have been problem free. He is MUCH more agreeable than ever. He is less emotional (a good thing), less cranky. He is remembering all the things he should. Overall his behavior is better. \r\nWe have tried many different medications and so far this is the most effective."',
  '"I used to take another oral contraceptive, which had 21 pill cycle, and was very happy-

In [92]:
for x in drug_dataset['train'].select(range(3)):
    print({"review": [html.unescape(o) for o in x["review"]]})

{'review': ['"', 'M', 'y', ' ', 's', 'o', 'n', ' ', 'i', 's', ' ', 'h', 'a', 'l', 'f', 'w', 'a', 'y', ' ', 't', 'h', 'r', 'o', 'u', 'g', 'h', ' ', 'h', 'i', 's', ' ', 'f', 'o', 'u', 'r', 't', 'h', ' ', 'w', 'e', 'e', 'k', ' ', 'o', 'f', ' ', 'I', 'n', 't', 'u', 'n', 'i', 'v', '.', ' ', 'W', 'e', ' ', 'b', 'e', 'c', 'a', 'm', 'e', ' ', 'c', 'o', 'n', 'c', 'e', 'r', 'n', 'e', 'd', ' ', 'w', 'h', 'e', 'n', ' ', 'h', 'e', ' ', 'b', 'e', 'g', 'a', 'n', ' ', 't', 'h', 'i', 's', ' ', 'l', 'a', 's', 't', ' ', 'w', 'e', 'e', 'k', ',', ' ', 'w', 'h', 'e', 'n', ' ', 'h', 'e', ' ', 's', 't', 'a', 'r', 't', 'e', 'd', ' ', 't', 'a', 'k', 'i', 'n', 'g', ' ', 't', 'h', 'e', ' ', 'h', 'i', 'g', 'h', 'e', 's', 't', ' ', 'd', 'o', 's', 'e', ' ', 'h', 'e', ' ', 'w', 'i', 'l', 'l', ' ', 'b', 'e', ' ', 'o', 'n', '.', ' ', 'F', 'o', 'r', ' ', 't', 'w', 'o', ' ', 'd', 'a', 'y', 's', ',', ' ', 'h', 'e', ' ', 'c', 'o', 'u', 'l', 'd', ' ', 'h', 'a', 'r', 'd', 'l', 'y', ' ', 'g', 'e', 't', ' ', 'o', 'u', 't', ' '

In [94]:
# this just keeps getting more horrible
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["review"], truncation=True)

In [95]:
%time tokenized_dataset = drug_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/138514 [00:00<?, ? examples/s]

Map:   0%|          | 0/46108 [00:00<?, ? examples/s]

CPU times: user 58 s, sys: 1.15 s, total: 59.1 s
Wall time: 8.79 s


In [96]:
%time tokenized_dataset = drug_dataset.map(tokenize_function, batched=False)

Map:   0%|          | 0/138514 [00:00<?, ? examples/s]

Map:   0%|          | 0/46108 [00:00<?, ? examples/s]

CPU times: user 39.4 s, sys: 171 ms, total: 39.6 s
Wall time: 39.5 s


In [97]:
def tokenize_and_split(examples):
    return tokenizer(
        examples["review"],
        truncation=True,
        max_length=128,
        return_overflowing_tokens=True,
    )

In [98]:
result = tokenize_and_split(drug_dataset["train"][0])

In [99]:
result

{'input_ids': [[101, 107, 1422, 1488, 1110, 9079, 1194, 1117, 2223, 1989, 1104, 1130, 19972, 11083, 119, 1284, 1245, 4264, 1165, 1119, 1310, 1142, 1314, 1989, 117, 1165, 1119, 1408, 1781, 1103, 2439, 13753, 1119, 1209, 1129, 1113, 119, 1370, 1160, 1552, 117, 1119, 1180, 6374, 1243, 1149, 1104, 1908, 117, 1108, 1304, 172, 14687, 1183, 117, 1105, 7362, 1111, 2212, 129, 2005, 1113, 170, 2797, 1313, 1121, 1278, 12020, 113, 1304, 5283, 1111, 1140, 119, 114, 146, 1270, 1117, 3995, 1113, 6356, 2106, 1105, 1131, 1163, 1106, 6166, 1122, 1149, 170, 1374, 1552, 119, 3969, 1293, 1119, 1225, 1120, 1278, 117, 1105, 1114, 2033, 1146, 1107, 1103, 2106, 119, 1109, 1314, 1160, 1552, 1138, 1151, 2463, 1714, 119, 1124, 1110, 150, 21986, 3048, 1167, 5340, 1895, 1190, 1518, 102], [101, 119, 1124, 1110, 1750, 6438, 113, 170, 1363, 1645, 114, 117, 1750, 172, 14687, 1183, 119, 1124, 1110, 11566, 1155, 1103, 1614, 1119, 1431, 119, 8007, 1117, 4658, 1110, 1618, 119, 1284, 1138, 1793, 1242, 1472, 23897, 1105, 117

In [100]:
[len(inp) for inp in result["input_ids"]]

[128, 49]

In [101]:
tokenized_dataset = drug_dataset.map(
    tokenize_and_split, batched=True, remove_columns=drug_dataset["train"].column_names
)

Map:   0%|          | 0/138514 [00:00<?, ? examples/s]

Map:   0%|          | 0/46108 [00:00<?, ? examples/s]

In [102]:
len(tokenized_dataset["train"]), len(drug_dataset["train"])

(206772, 138514)

In [106]:
print(tokenized_dataset["train"]['overflow_to_sample_mapping'])

[0, 0, 1, 1, 2, 3, 3, 4, 5, 5, 6, 7, 7, 8, 8, 9, 10, 11, 11, 12, 12, 13, 13, 14, 14, 15, 15, 16, 16, 17, 18, 18, 19, 19, 20, 21, 21, 22, 23, 24, 24, 25, 25, 26, 27, 27, 28, 29, 30, 31, 32, 32, 33, 34, 35, 35, 36, 37, 37, 38, 39, 40, 40, 41, 42, 42, 43, 44, 45, 45, 45, 45, 45, 46, 47, 47, 48, 49, 50, 51, 52, 52, 53, 53, 54, 54, 55, 55, 56, 56, 57, 58, 58, 59, 59, 60, 61, 62, 63, 64, 64, 65, 66, 66, 67, 67, 68, 68, 69, 70, 71, 71, 72, 73, 73, 74, 74, 75, 76, 77, 77, 78, 78, 79, 79, 80, 80, 81, 81, 82, 83, 83, 84, 84, 85, 85, 86, 87, 87, 88, 89, 89, 90, 91, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 101, 102, 102, 103, 104, 105, 105, 106, 106, 107, 107, 108, 108, 109, 110, 111, 112, 112, 113, 113, 114, 114, 115, 116, 117, 117, 118, 118, 119, 119, 120, 120, 121, 122, 122, 123, 123, 124, 125, 126, 126, 127, 128, 129, 129, 130, 130, 131, 131, 132, 133, 133, 134, 135, 135, 136, 137, 137, 138, 139, 139, 140, 141, 142, 143, 144, 145, 145, 146, 146, 147, 148, 149, 150, 151, 151, 152, 152, 153

In [107]:
def tokenize_and_split(examples):
    result = tokenizer(
        examples["review"],
        truncation=True,
        max_length=128,
        return_overflowing_tokens=True,
    )
    # Extract mapping between new and old indices
    sample_map = result.pop("overflow_to_sample_mapping")
    for key, values in examples.items():
        result[key] = [values[i] for i in sample_map]
    return result

In [108]:
tokenized_dataset = drug_dataset.map(tokenize_and_split, batched=True)
tokenized_dataset

Map:   0%|          | 0/138514 [00:00<?, ? examples/s]

Map:   0%|          | 0/46108 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 206772
    })
    test: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 68876
    })
})

In [109]:
drug_dataset.set_format('pandas')

In [110]:
drug_dataset['train'][:3]

,patient_id,drugName,condition,review,rating,date,usefulCount,review_length
0,95260,Guanfacine,adhd,"""My son is halfway through his fourth week of ...",8.0,"April 27, 2010",192,141
1,92703,Lybrel,birth control,"""I used to take another oral contraceptive, wh...",5.0,"December 14, 2009",17,134
2,138000,Ortho Evra,birth control,"""This is my first time using any form of birth...",8.0,"November 3, 2015",10,89


In [111]:
train_df = drug_dataset["train"][:]

In [112]:
type(train_df)

pandas.core.frame.DataFrame

In [113]:
type(drug_dataset["train"])

datasets.arrow_dataset.Dataset

In [114]:
frequencies = (
    train_df["condition"]
    .value_counts()
    .to_frame()
    .reset_index()
    .rename(columns={"index": "condition", "count": "frequency"})
)
frequencies.head()

,condition,frequency
0,birth control,27655
1,depression,8023
2,acne,5209
3,anxiety,4991
4,pain,4744


In [116]:
from datasets import Dataset

freq_dataset = Dataset.from_pandas(frequencies)
freq_dataset

Dataset({
    features: ['condition', 'frequency'],
    num_rows: 819
})

In [117]:
drug_dataset.reset_format()

In [118]:
# we are going to create a validation set!
# I already knew how to do this  
drug_dataset_clean = drug_dataset["train"].train_test_split(train_size=0.8, seed=42)
# Rename the default "test" split to "validation"
drug_dataset_clean["validation"] = drug_dataset_clean.pop("test")
# Add the "test" set to our `DatasetDict`
drug_dataset_clean["test"] = drug_dataset["test"]
drug_dataset_clean

DatasetDict({
    train: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 110811
    })
    validation: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 27703
    })
    test: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 46108
    })
})

In [119]:
drug_dataset_clean.save_to_disk("./data/drug-reviews")

Saving the dataset (0/1 shards):   0%|          | 0/110811 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/27703 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/46108 [00:00<?, ? examples/s]

In [120]:
from datasets import load_from_disk

drug_dataset_reloaded = load_from_disk("./data/drug-reviews")
drug_dataset_reloaded

DatasetDict({
    train: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 110811
    })
    validation: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 27703
    })
    test: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 46108
    })
})